<a href="https://colab.research.google.com/github/singhmegh/Scrap-Simulation-Challenge/blob/main/Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mostafaabla/garbage-classification

Dataset URL: https://www.kaggle.com/datasets/mostafaabla/garbage-classification
License(s): ODbL-1.0
 62% 149M/239M [00:00<00:00, 1.56GB/s]
100% 239M/239M [00:00<00:00, 910MB/s] 


In [ ]:
# extract the compressed dataset
from zipfile import ZipFile
dataset = "/content/garbage-classification.zip"

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
!ls

garbage_classification	garbage-classification.zip  kaggle.json  sample_data


In [ ]:
!ls garbage_classification

battery     brown-glass  clothes      metal  plastic  trash
biological  cardboard	 green-glass  paper  shoes    white-glass


**Import Dependencies**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# let's prepare the data and generate the data

from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen_train = ImageDataGenerator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2,
                               brightness_range = (0.1, 0.5), horizontal_flip=True)

train_data = gen_train.flow_from_directory("/content/garbage_classification",
                                           target_size = (224, 224), batch_size = 32, class_mode="categorical")

Found 15515 images belonging to 12 classes.


In [ ]:
!pip install torch torchvision torchaudio
!pip install scikit-learn

**Dependenices**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import os

**Data Transforms**

In [ ]:
import os, shutil
from sklearn.model_selection import train_test_split

DATA_DIR = "/content/garbage_classification"
BASE_DIR = "/content/garbage_split"

for split in ["train", "test"]:
    split_path = os.path.join(BASE_DIR, split)
    if not os.path.exists(split_path):
        os.makedirs(split_path)


classes = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]

for cls in classes:
    cls_path = os.path.join(DATA_DIR, cls)
    images = os.listdir(cls_path)

    # Split 80/20
    train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)

    for img_set, split in [(train_imgs, "train"), (test_imgs, "test")]:
        split_cls_path = os.path.join(BASE_DIR, split, cls)
        os.makedirs(split_cls_path, exist_ok=True)
        for img in img_set:
            src = os.path.join(cls_path, img)
            dst = os.path.join(split_cls_path, img)
            shutil.copy(src, dst)

print("Dataset split into train/test at:", BASE_DIR)

Dataset split into train/test at: /content/garbage_split


In [ ]:
train_dataset = torchvision.datasets.ImageFolder(os.path.join(BASE_DIR, "train"), transform=train_transform)
test_dataset  = torchvision.datasets.ImageFolder(os.path.join(BASE_DIR, "test"), transform=test_transform)

In [ ]:
DATA_DIR = "/content/garbage_split"

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_transform)
test_dataset  = torchvision.datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

classes = train_dataset.classes
print("Classes:", classes)

Classes: ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']


**Model (ResNet-18 + Transfer Learning**)

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False


model.fc = nn.Linear(model.fc.in_features, len(classes))

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 161MB/s]


**Training the data**

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    return model

model = train_model(model, train_loader, criterion, optimizer, epochs=5)

Epoch [1/5], Loss: 0.7295
Epoch [2/5], Loss: 0.3973
Epoch [3/5], Loss: 0.3425
Epoch [4/5], Loss: 0.3137
Epoch [5/5], Loss: 0.2988


**Evaluation**

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    print(classification_report(y_true, y_pred, target_names=classes))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

evaluate(model, test_loader)

              precision    recall  f1-score   support

     battery       0.85      0.97      0.91       189
  biological       0.95      0.97      0.96       197
 brown-glass       0.89      0.94      0.92       122
   cardboard       0.94      0.83      0.88       179
     clothes       0.99      0.98      0.98      1065
 green-glass       0.94      0.90      0.92       126
       metal       0.85      0.82      0.84       154
       paper       0.90      0.83      0.87       210
     plastic       0.84      0.75      0.79       173
       shoes       0.92      0.99      0.95       396
       trash       0.89      0.89      0.89       140
 white-glass       0.84      0.85      0.85       155

    accuracy                           0.93      3106
   macro avg       0.90      0.89      0.90      3106
weighted avg       0.93      0.93      0.93      3106

Confusion Matrix:
 [[ 184    0    0    2    0    0    1    0    0    0    2    0]
 [   0  191    1    0    0    0    0    0    0    5

In [ ]:
torch.save(model.state_dict(), "resnet18_scrap.pth")

**Export to TorchScript**

In [ ]:
model_cpu = model.cpu()
dummy_input = torch.randn(1, 3, 224, 224)

traced_model = torch.jit.trace(model_cpu, dummy_input)
traced_model.save("scrap_resnet18.pt")
print("TorchScript model saved")

TorchScript model saved


**Export to ONNX**

In [ ]:
!pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(
    model.cpu(),
    dummy_input,
    "garbage_resnet18.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
)

print("ONNX model saved")

/tmp/ipython-input-4084839472.py:3: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


ONNX model saved


**Lightweight Inference Script**

In [ ]:
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F

In [ ]:
model = torch.jit.load("scrap_resnet18.pt")
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Class names (12 classes in Garbage Classification dataset)
classes = [
    "cardboard", "glass", "metal", "paper", "plastic",
    "trash", "biological", "green-glass", "brown-glass",
    "white-glass", "clothes", "other"
]

def predict_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img_t = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(img_t)
        probs = F.softmax(outputs, dim=1)[0]
        conf, pred = torch.max(probs, dim=0)
    return classes[pred.item()], conf.item()


img_path = "/content/garbage_split/test/cardboard/cardboard103.jpg"
label, confidence = predict_image(img_path)
print(f"Predicted: {label} | Confidence: {confidence:.2f}")

Predicted: paper | Confidence: 0.99


**Dummy Conveyor Simulatio**n

In [ ]:
import os
import time
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
MODEL_PATH = "scrap_resnet18.pt"
IMAGE_FOLDER = "/content/garbage_split/test/battery"
CONF_THRESHOLD = 0.6
RESULTS_CSV = "conveyor_results.csv"

In [ ]:
model = torch.jit.load(MODEL_PATH)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [ ]:
classes = [
    "cardboard", "glass", "metal", "paper", "plastic",
    "trash", "biological", "green-glass", "brown-glass",
    "white-glass", "clothes", "other"
]
def classify_frame(img_path):
    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = model(img_t)
        probs = F.softmax(outputs, dim=1)[0]
        conf, pred = torch.max(probs, dim=0)
    label = classes[pred.item()]
    flag = "LOW CONFIDENCE" if conf.item() < CONF_THRESHOLD else ""
    return label, conf.item(), flag
results = []

for frame in sorted(os.listdir(IMAGE_FOLDER)):
    if not frame.lower().endswith((".jpg",".png",".jpeg")):
        continue
    frame_path = os.path.join(IMAGE_FOLDER, frame)

    label, conf, flag = classify_frame(frame_path)
    print(f"Frame: {frame} | Predicted: {label} | Confidence: {conf:.2f} {flag}")

    results.append({
        "frame": frame,
        "prediction": label,
        "confidence": conf,
        "flag": flag
    })

    time.sleep(1)

pd.DataFrame(results).to_csv(RESULTS_CSV, index=False)
print(f"Simulation complete. Results saved to {RESULTS_CSV}")

Frame: battery10.jpg | Predicted: cardboard | Confidence: 0.99 
Frame: battery101.jpg | Predicted: cardboard | Confidence: 0.96 
Frame: battery103.jpg | Predicted: cardboard | Confidence: 0.99 
Frame: battery104.jpg | Predicted: cardboard | Confidence: 1.00 
Frame: battery113.jpg | Predicted: cardboard | Confidence: 0.40 LOW CONFIDENCE
Frame: battery117.jpg | Predicted: cardboard | Confidence: 1.00 
Frame: battery118.jpg | Predicted: cardboard | Confidence: 1.00 
Frame: battery125.jpg | Predicted: cardboard | Confidence: 0.98 
Frame: battery128.jpg | Predicted: cardboard | Confidence: 0.77 
Frame: battery134.jpg | Predicted: cardboard | Confidence: 1.00 
Frame: battery136.jpg | Predicted: cardboard | Confidence: 1.00 
Frame: battery143.jpg | Predicted: cardboard | Confidence: 0.99 
Frame: battery151.jpg | Predicted: cardboard | Confidence: 0.99 
Frame: battery156.jpg | Predicted: cardboard | Confidence: 1.00 
Frame: battery157.jpg | Predicted: cardboard | Confidence: 1.00 
Frame: batte